Primer paso: "Las importaciones necesarias"  

- sys: se utiliza para modificar el sys.path, permitiendo que Python encuentre módulos personalizados que están fuera del directorio actual del notebook.

- csv: módulo estándar de Python que permite leer y escribir archivos CSV.

- Path de pathlib: para trabajar genericamente con rutas de archivos. Esto es importante porque este proyecto requiere rutas configuradas a través de variables definidas en src/util/constantes.py, para que cumpla con una estructura flexible que se pueda ejecutar en cualquier maquina de manera consistente sin necesidad de escribir rutas absolutas ni relativas manualmente. 

- Se importan dos rutas definidas en constantes.py

- Se importa la función fusionar_csv definida en fusionador.py

In [ ]:
import sys
import csv
from pathlib import Path

sys.path.append("..") # Esto es necesario porque el archivo constantes.py están en src/util/, que está fuera del directorio notebooks/.
from src.util.constantes import FILES_PATH, FUSION_PATH
from src.util.fusionador import fusionar_csv


Fusión de los archivos de individuos en un solo archivo 

En esta sección se invoca la función fusionar_csv definida en src/util/fusionador.py para buscar y unir todos los archivos usu_individual_* encontrados en las subcarpetas de files_eph/. El archivo resultante se guarda en la carpeta fusion_eph/ generando el dataset unificado de individuos.

In [ ]:
archivo_salida_indiv = FUSION_PATH / "individuos_fusionado.csv"

# Ejecuta la fusión de todos los archivos 'usu_individual_*'
fusionar_csv("usu_individual_", FILES_PATH, archivo_salida_indiv)

Fusión de los archivos de individuos en un solo archivo

En esta sección se invoca la función fusionar_csv definida en src/util/fusionador.py para buscar y unir todos los archivos usu_individual* encontrados en las subcarpetas de files_eph/. El archivo resultante se guarda en la carpeta fusion_eph/ generando el dataset unificado de individuos.

In [ ]:
# Carga de archivo fusionado 
with archivo_salida_indiv.open(encoding="utf-8") as f:
    lector = csv.DictReader(f, delimiter=";")
    datos_indiv = list(lector)

Se traducen los valores CH04 numéricos a "Masculino" y "Femenino". El resultado se almacena en una nueva columna llamada CH04_str.  

In [ ]:
from src.util.modulos_seccion_a import punto3

punto3(datos_indiv)

Se traducen los valores NIVEL_ED numéricos a descripciones en formato texto. 
El resultado se debe almacenar en una nueva columna llamada NIVEL_ED_str. La 
transformación debe seguir las siguientes reglas: 
1: "Primario incompleto". 
2: "Primario completo". 
3 "Secundario incompleto". 
4: "Secundario completo". 
5 a 6: "Superior o universitario". 
7 o 9: “Sin informacion”. 

In [ ]:
from src.util.modulos_seccion_a import punto4

punto4(datos_indiv)

Este módulo procesa los datos individuales para agregar dos columnas importantes: CONDICION_LABORAL y UNIVERSITARIO. La columna CONDICION_LABORAL clasifica el estado laboral de cada individuo basado en su ESTADO y CAT_OCUP. Mientras tanto, la columna UNIVERSITARIO determina si un individuo ha completado estudios universitarios superiores, utilizando la edad y el nivel educativo proporcionados en los datos de entrada.

In [ ]:
from src.util.modulos_seccion_a import punto5_6

punto5_6(datos_indiv)

Para que se guarden los cambios hechos, "individuos_fusionado_actualizado.csv" este archivo va a mostrar las nuevas columnas en el Excel.

In [ ]:

from src.util.modulos_seccion_a import actualizar_archivo

ruta = FUSION_PATH / "individuos_fusionado_actualizado.csv"

actualizar_archivo(datos_indiv, ruta)

Importo la funcion calcular_porcentaje_lectura_escritura que calculo e informo el porcentaje de personas mayores a 2 años capaces e incapaces de leer y escribir.

In [ ]:
from src.util.modulos_seccion_b import calcular_porcentaje_lectura_escritura
archivo_actualizado_individuos = FUSION_PATH / "individuos_fusionado_actualizado.csv"
# Calcular y mostrar resultados
resultados_por_anio = calcular_porcentaje_lectura_escritura(datos_indiv)
print("Porcentaje de personas mayores de 2 años que pueden y no pueden leer y escribir")
for anio, porcentajes in resultados_por_anio.items():
    print(f"Año {anio}:")
    print(f"  Pueden leer y escribir: {porcentajes['porcentaje_pueden_leer_escribir']:.2f}%")
    print(f"  No pueden leer ni escribir: {porcentajes['porcentaje_no_pueden_leer_escribir']:.2f}%")

In [ ]:
from src.util.modulos_seccion_b import total_universitarios
from src.util.modulos_seccion_b import tabla_nivel_estudios


A partir del archivo individuos_fusionado_actualizado.csv, informo para cada aglomerado el porcentaje de personas que hayan cursado al menos en nivel universitario o superior.

In [ ]:
total_universitarios(ruta)

Retorna una tabla que contenga la cantidad de personas mayores de edad según su nivel de estudios alcanzados.


In [ ]:
tabla_nivel_estudios(ruta)

En esta sección se calcula el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior, para un año y trimestre específicos seleccionados por el usuario. Se utiliza la función `porcentaje_extranjeros_universitarios` definida en `src/util/modulos.py`. Esta función toma como entrada el dataset fusionado de individuos actualizado (`individuos_fusionado_actualizado.csv`) y realiza el cálculo considerando la columna `PONDERA` para ponderar correctamente los datos."

In [ ]:
from src.util.modulos_seccion_b import porcentaje_extranjeros_universitarios

año_consulta = int(input("Ingrese el año a consultar (entre 2020 y 2024): "))
if año_consulta < 2020 or año_consulta > 2024:
    print("Año no válido. Debe estar entre 2016 y 2024.")
else:
    trimestre_consulta = int(input("Ingrese el trimestre a consultar (1, 2, 3 o 4): "))
    if trimestre_consulta < 1 or trimestre_consulta > 4:
        print(" Trimestre no válido. Debe ser 1, 2, 3 o 4.")
    else:
        resultado = porcentaje_extranjeros_universitarios(datos_indiv, año_consulta, trimestre_consulta)

        print(f"\n Porcentaje de extranjeros universitarios en {año_consulta}T{trimestre_consulta}: {resultado}%")

Se determina el año y trimestre con menor desocupación utilizando la función menor_desocupacion, que identifica a los desocupados a partir de la columna CONDICION_LABORAL y ajusta los resultados con la columna PONDERA.

In [ ]:
from src.util.modulos_seccion_b import menor_desocupacion

# Calcular el año y trimestre con menor desocupación
resultado = menor_desocupacion(datos_indiv)

# Informar el resultado
if resultado != (0, 0):
    print(f"El año y trimestre con menor desocupación es: Año {resultado[0]}, Trimestre {resultado[1]}")
else:
    print("No se encontraron datos válidos para calcular el período con menor desocupación.")

En esta sección se solicita al usuario que ingrese el código de dos aglomerados, y se imprime el porcentaje de personas mayores que no completaron el secundario. Se utiliza la función tabla_año_trimestre_aglomerado, definida en src/util/modulos. Esta función recibe como entrada el dataset y los aglomerados introducidos por el usuario, y realiza filtros para calcular los porcentajes.

In [ ]:
from src.util.modulos_seccion_b import tabla_año_trimestre_aglomerado
#Solicito al Usuario que ingrese los codigos de los aglomerados
aglo1= input("Ingrese el codigo del primer  aglomerado")
aglo2= input("Ingrese el codigo del segundo aglomerado")
#Llamo a la funcion para imprimir la tabla
tabla_año_trimestre_aglomerado(datos_indiv,aglo1,aglo2)

In [ ]:
from src.util.constantes import FUSION_PATH
archivo_salida_hogar = FUSION_PATH / "hogares_fusionado.csv"
with archivo_salida_hogar.open(encoding="utf-8") as f:
    lector = csv.DictReader(f, delimiter=";")
    datos_hogar = list(lector)

En esta sección se calcula, para cada aglomerado, el porcentaje de jubilados que viven en una vivienda con condiciones habitacionales insuficientes. Se invoca la función calcular_porcentaje_jubilados_habitabilidad, que se encuentra en src/util/modulos.py. Esta función recibe como entrada los datasets de hogares e individuos y devuelve un diccionario cuya clave es el nombre del aglomerado y cuyo valor es el porcentaje correspondiente.

In [ ]:
from src.util.modulos_seccion_b import calcular_porcentaje_jubilados_habitalidad

datos=calcular_porcentaje_jubilados_habitalidad(datos_hogar,datos_indiv)


En esta sección se solicita al usuario que ingrese un año para imprimir la cantidad de personas con estudios universitarios o superiores que viven en hogares con condiciones habitacionales insuficientes. Se invoca la función cant_uni_habitabilidad, que se encuentra en src/util/modulos. Esta función recibe como entrada el dataset de hogares, el de individuos y el año ingresado por el usuario.

In [ ]:
from src.util.modulos_seccion_b import cant_uni_habitalidad

año=input('Introduzca El Año Para Buscar la cantidad de universitarios o educacion mayor en condiciones insuficientes de vivienda ')#Intruducir 2023 o 2024 porque son los años en el dataset que tienen 4to trimestre

cant_uni_habitalidad(datos_hogar,datos_indiv,año)